In [11]:
import os
import json
import easyocr
import pandas as pd
from pathlib import Path
from nltk import word_tokenize

In [12]:
train_images = sorted(list(Path('images').glob("*/*.png")))
val_images = [os.path.join(os.getcwd(), 'Test Images', img) for img in os.listdir(os.path.join(os.getcwd(), 'Test Images'))]

In [13]:
reader = easyocr.Reader(['en'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [14]:
def remove_whitespace(line_lower):
    return  " ".join(line_lower.split())

In [15]:
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
            
    return result

In [16]:
from nltk.tokenize import RegexpTokenizer

def remove_punct(text):
    
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

def lemmatization(text):
    
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))
    
    return result

In [21]:
def image_to_text(image_paths):

    for image_path in image_paths:
        if image_path.endswith('.png'):
            ocr_result = reader.readtext(str(image_path), batch_size=16)

            ocr_page = ""
            for bbox, word, confidance in ocr_result:
                ocr_page += word + " "

            with Path(str(image_path).replace('.png', '.txt')).open('w') as f:
                line_lower = ocr_page.lower()
                line_without_space = remove_whitespace(line_lower)
                token_list = word_tokenize(line_without_space)
                clean_text = remove_stopwords(token_list)
                no_punct = remove_punct(clean_text)
                lemma_text = lemmatization(no_punct)
                clean_text = " ".join(lemma_text)

                # json.dump(clean_text[1:-1], f)
                # text_file = open(os.path.join(text_folder, "data.txt"), 'w')
                f.write(clean_text)
                f.close()

In [22]:
image_to_text(train_images)

In [23]:
image_to_text(val_images)